In [ ]:
import tensorflow as tf
import pickle
import numpy as np


def load(data_path):
    with open(data_path,'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

class MnistData:
    def __init__(self, filenames, need_shuffle, datatype='training'):
        all_data = []
        all_labels = []
        x_train, y_train, x_test, y_test = load(filenames) #"data/mnist.pkl"
        
        if datatype=='training':
            self._data = x_train / 127.5 -1
            self._labels = y_train
            print(self._data.shape)
            print(self._labels.shape)
        else:
            self._data = x_test / 127.5 -1
            self._labels = y_test
            print(self._data.shape)
            print(self._labels.shape)
            
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

In [ ]:
train_filenames = "../4_basic_image_recognition/data/mnist.pkl"
train_data = MnistData(train_filenames, True, 'training')
test_data = MnistData(train_filenames, False, 'test')

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)  
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # ksize [1,x,y,1]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
batch_size = 100
n_batch = train_data._num_examples // batch_size

x = tf.placeholder(tf.float32, shape=[None, 28*28])
y = tf.placeholder(tf.int64, shape=[None])

x_image = tf.reshape(x, [-1, 28, 28, 1])

# first convolutional layer
W_conv_1 = weight_variable([5, 5, 1, 32])
b_conv_1 = bias_variable([32])
h_conv_1 = tf.nn.relu(tf.add(conv2d(x_image, W_conv_1), b_conv_1))
h_pool_1 = max_pool_2x2(h_conv_1)

# second convolutional layer
W_conv_2 = weight_variable([5, 5, 32, 64])
b_conv_2 = bias_variable([64])
h_conv_2 = tf.nn.relu(tf.add(conv2d(h_pool_1, W_conv_2), b_conv_2))
h_pool_2 = max_pool_2x2(h_conv_2)

# fully connected layer 1
h_pool_2_flat = tf.reshape(h_pool_2, [-1, 7*7*64])
W_fc_1 = weight_variable([7*7*64, 1024])
b_fc_1 = bias_variable([1024])
h_fc_1 = tf.nn.relu(tf.add(tf.matmul(h_pool_2_flat, W_fc_1), b_fc_1))

# fully connected layer 2
W_fc_2 = weight_variable([1024, 10])
b_fc_2 = bias_variable([10])
y_ = tf.add(tf.matmul(h_fc_1, W_fc_2), b_fc_2)

#prediction = tf.nn.softmax(tf.add(tf.matmul(h_fc_1, W_fc_2), b_fc_2))
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
#train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))  # 
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
#train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss) # Acc by step 1: 8%, step2: 10% ...
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss) # Acc by step 1: 94%, step2: 96% ...
predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(20):
        for batch in range(n_batch):
            batch_xs, batch_ys = train_data.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})
        
        test_images, test_labels = test_data.next_batch(500)
        acc = sess.run(accuracy, feed_dict={x: test_images, y: test_labels})
        print("Epoch: " + str(epoch + 1) + "=> Testing Accuracy=" + str(acc))

